In [1]:
import sys
sys.path.insert(0,'/home/olgan96/.env/lib/python3.11/site-packages/')
import pandas as pd
import numpy as np

### mapping genes

In [43]:
matrix = pd.read_parquet('../../NO_BACKUP/tahoe/sig_matrix.parquet')

In [2]:
df_lincs = pd.read_parquet('../../NO_BACKUP/lincs_2020/signature_response_features_r2_top0.7_final.parquet')

In [3]:
df_lincs = df_lincs.iloc[0]

In [79]:
df_sig_genes = pd.read_parquet('../../NO_BACKUP/tahoe/sig_genes.parquet')

In [80]:
df_sig_genes

,gene_name
0,GAS8-AS1
1,ATXN8OS
2,XIST
3,INE1
4,FAM30A
...,...
12119,ARMCX4
12120,NBPF10
12121,TIMM23
12122,ZNF783


In [81]:
df_sig_genes = df_sig_genes.reset_index().rename(columns={'index': 'tahoe_gene_id', 'gene_name': 'gene_symbol'})

In [82]:
df_sig_genes['tahoe_gene_id'] = df_sig_genes['tahoe_gene_id'].astype(int).astype(str)

In [83]:
lincs_genes = pd.read_parquet('../../NO_BACKUP/lincs_2020/genes_info.parquet')

In [84]:
lincs_genes = lincs_genes[['gene_id', 'gene_symbol', 'ensembl_id', 'gene_title']].rename(columns={'gene_id': 'lincs_gene_id'})

In [85]:
lincs_genes['lincs_gene_id'] = lincs_genes['lincs_gene_id'].astype(int).astype(str)

In [86]:
lincs_genes = lincs_genes.merge(df_sig_genes, on='gene_symbol', how='left')

In [87]:
lincs_genes

,lincs_gene_id,gene_symbol,ensembl_id,gene_title,tahoe_gene_id
0,750,GAS8-AS1,ENSG00000221819,GAS8 antisense RNA 1,0
1,6315,ATXN8OS,None,ATXN8 opposite strand lncRNA,1
2,7503,XIST,ENSG00000229807,X inactive specific transcript,2
3,8552,INE1,ENSG00000224975,inactivation escape 1,3
4,9834,FAM30A,ENSG00000226777,family with sequence similarity 30 member A,4
...,...,...,...,...,...
12323,100287932,TIMM23,ENSG00000265354,translocase of inner mitochondrial membrane 23,12121
12324,100289678,ZNF783,ENSG00000204946,zinc finger family member 783,12122
12325,100507436,MICA,ENSG00000204520,MHC class I polypeptide-related sequence A,12123
12326,9142,TMEM257,ENSG00000221870,transmembrane protein 257,NaN


In [77]:
lincs_genes_order = df_lincs.columns[df_lincs.columns.isin(lincs_genes['lincs_gene_id'])]

In [93]:
matrix_t = matrix.T.reset_index()

In [94]:
matrix_t = matrix_t.rename(columns={'index': 'tahoe_gene_id'})

In [96]:
matrix_t['tahoe_gene_id'] = matrix_t['tahoe_gene_id'].astype(str)

In [98]:
genes = lincs_genes.merge(matrix_t, on='tahoe_gene_id', how='left')

In [100]:
genes = genes.drop(columns=['gene_symbol', 'ensembl_id', 'gene_title', 'tahoe_gene_id'])

In [103]:
genes = genes.set_index('lincs_gene_id')

In [110]:
genes = genes.loc[lincs_genes_order]

In [112]:
#genes.reset_index().to_parquet('../../NO_BACKUP/tahoe/genes_tahoe_lincs.parquet', index=False)

In [113]:
matrix_mapped = genes.T

In [116]:
matrix_mapped

,10,100,1000,10000,10001,10003,10004,10005,10006,10007,...,9985,9987,9988,9989,999,9990,9991,9992,9993,9997
0,0.0,1.132214,2.268551,3.580747,0.182860,0.276262,0.000000,-0.428292,0.953880,-1.164150,...,1.842929,0.366734,-0.335411,0.782379,-0.359100,-0.234669,2.092593,0.0,-0.832242,1.058628
1,0.0,0.503395,2.343700,2.903499,0.115342,0.302952,0.092239,-0.005663,0.404231,-0.627971,...,2.046573,0.165296,-0.113157,0.101338,-0.583370,-0.234678,1.734972,0.0,-0.535040,0.692996
2,0.0,0.351372,2.447946,3.701025,-0.293082,0.000000,0.000000,-0.692030,0.638024,-0.865866,...,2.075660,0.152147,0.160935,0.444318,-0.384877,-0.837646,2.344321,0.0,-0.937542,0.649613
3,0.0,0.899534,2.783774,3.010512,-0.008444,0.300268,0.083557,-0.355955,0.560204,-0.627576,...,1.366750,0.400315,0.087598,0.663222,0.103934,-0.031691,2.111688,0.0,-1.267154,0.513679
4,0.0,1.173077,2.287166,3.536037,-0.353265,0.207164,0.000000,-0.705312,0.567079,-1.372054,...,1.823357,-0.357533,-0.097746,0.487961,-1.207489,-0.640630,2.030661,0.0,-1.128126,0.567049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14437,0.0,0.698976,1.980942,0.560504,1.495647,0.000000,0.109241,-0.372341,0.465674,-2.114707,...,1.466497,-0.400262,-0.439058,0.239185,-0.017977,-0.018577,0.836884,0.0,-0.680973,0.757680
14438,0.0,-0.079430,2.326757,0.146025,1.252163,0.000000,0.000000,-1.159605,0.163694,-2.019975,...,1.519747,0.084264,-1.421381,0.510383,-0.330845,-0.875411,0.857085,0.0,-1.507406,0.356137
14439,0.0,0.421218,1.563652,0.591608,1.014364,0.172848,0.000000,-0.476005,0.162891,-1.512191,...,1.030763,-0.424219,-1.296579,0.196420,-0.756021,0.250010,0.719426,0.0,-1.023503,0.072118
14440,0.0,-0.077913,1.333669,0.467294,1.233823,0.389567,0.000000,-0.644001,0.687567,-1.936865,...,1.546654,-0.181080,-0.800917,-0.022180,-0.940195,0.395158,1.009031,0.0,-1.189201,0.329604


# map to sig

In [117]:
df_sig = pd.read_parquet('../../NO_BACKUP/tahoe/computed_signaturs_meta.parquet')

In [118]:
df_sig

,cl_sm_dose
0,CVCL_0023_APTO-253_0.05
1,CVCL_0023_AT7519_0.05
2,CVCL_0023_AZD-7648_0.05
3,CVCL_0023_AZD-8055_0.05
4,CVCL_0023_AZD1390_0.05
...,...
14437,CVCL_1731_Uridine_0.5
14438,CVCL_1731_Valdecoxib_0.5
14439,CVCL_1731_Vandetanib_0.5
14440,CVCL_1731_Vismodegib_0.5


In [121]:
df = pd.concat([df_sig, matrix_mapped], axis=1)

In [122]:
df

,cl_sm_dose,10,100,1000,10000,10001,10003,10004,10005,10006,...,9985,9987,9988,9989,999,9990,9991,9992,9993,9997
0,CVCL_0023_APTO-253_0.05,0.0,1.132214,2.268551,3.580747,0.182860,0.276262,0.000000,-0.428292,0.953880,...,1.842929,0.366734,-0.335411,0.782379,-0.359100,-0.234669,2.092593,0.0,-0.832242,1.058628
1,CVCL_0023_AT7519_0.05,0.0,0.503395,2.343700,2.903499,0.115342,0.302952,0.092239,-0.005663,0.404231,...,2.046573,0.165296,-0.113157,0.101338,-0.583370,-0.234678,1.734972,0.0,-0.535040,0.692996
2,CVCL_0023_AZD-7648_0.05,0.0,0.351372,2.447946,3.701025,-0.293082,0.000000,0.000000,-0.692030,0.638024,...,2.075660,0.152147,0.160935,0.444318,-0.384877,-0.837646,2.344321,0.0,-0.937542,0.649613
3,CVCL_0023_AZD-8055_0.05,0.0,0.899534,2.783774,3.010512,-0.008444,0.300268,0.083557,-0.355955,0.560204,...,1.366750,0.400315,0.087598,0.663222,0.103934,-0.031691,2.111688,0.0,-1.267154,0.513679
4,CVCL_0023_AZD1390_0.05,0.0,1.173077,2.287166,3.536037,-0.353265,0.207164,0.000000,-0.705312,0.567079,...,1.823357,-0.357533,-0.097746,0.487961,-1.207489,-0.640630,2.030661,0.0,-1.128126,0.567049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14437,CVCL_1731_Uridine_0.5,0.0,0.698976,1.980942,0.560504,1.495647,0.000000,0.109241,-0.372341,0.465674,...,1.466497,-0.400262,-0.439058,0.239185,-0.017977,-0.018577,0.836884,0.0,-0.680973,0.757680
14438,CVCL_1731_Valdecoxib_0.5,0.0,-0.079430,2.326757,0.146025,1.252163,0.000000,0.000000,-1.159605,0.163694,...,1.519747,0.084264,-1.421381,0.510383,-0.330845,-0.875411,0.857085,0.0,-1.507406,0.356137
14439,CVCL_1731_Vandetanib_0.5,0.0,0.421218,1.563652,0.591608,1.014364,0.172848,0.000000,-0.476005,0.162891,...,1.030763,-0.424219,-1.296579,0.196420,-0.756021,0.250010,0.719426,0.0,-1.023503,0.072118
14440,CVCL_1731_Vismodegib_0.5,0.0,-0.077913,1.333669,0.467294,1.233823,0.389567,0.000000,-0.644001,0.687567,...,1.546654,-0.181080,-0.800917,-0.022180,-0.940195,0.395158,1.009031,0.0,-1.189201,0.329604


In [180]:
df_sig_resp = pd.read_parquet('../../NO_BACKUP/tahoe/signature_response.parquet')

In [181]:
df_sig_resp = df_sig_resp.drop_duplicates('cl_sm_dose')

In [179]:
df = df.drop_duplicates('cl_sm_dose')

In [182]:
df_sig_resp

,index,cl_sm_dose,cell_line,drug,dose,pubchem_cid,cell_line_drug,responses
0,0,CVCL_0023_APTO-253_0.05,CVCL_0023,APTO-253,0.05,11960271,CVCL_0023_11960271,0.989992
1,1,CVCL_0023_AZD-7648_0.05,CVCL_0023,AZD-7648,0.05,135151360,CVCL_0023_135151360,0.991688
2,2,CVCL_0023_AZD-8055_0.05,CVCL_0023,AZD-8055,0.05,25262965,CVCL_0023_25262965,0.689011
3,3,CVCL_0023_AZD1390_0.05,CVCL_0023,AZD1390,0.05,126689157,CVCL_0023_126689157,0.999031
4,4,CVCL_0023_Afatinib_0.05,CVCL_0023,Afatinib,0.05,10184653,CVCL_0023_10184653,0.912780
...,...,...,...,...,...,...,...,...
4381,7978,CVCL_1731_5-Fluorouracil_0.5,CVCL_1731,5-Fluorouracil,0.50,3385,CVCL_1731_3385,0.923447
4382,7981,CVCL_1731_Belinostat_0.5,CVCL_1731,Belinostat,0.50,6918638,CVCL_1731_6918638,0.912687
4383,7990,CVCL_1731_Fulvestrant_0.5,CVCL_1731,Fulvestrant,0.50,104741,CVCL_1731_104741,0.901148
4384,7997,CVCL_1731_Panobinostat_0.5,CVCL_1731,Panobinostat,0.50,6918837,CVCL_1731_6918837,0.470417


In [183]:
df_comp = pd.read_parquet('../../NO_BACKUP/tahoe/comp_overlap_fps.parquet')

In [184]:
df_comp = df_comp.drop(columns='drug')
df_comp = df_comp.drop_duplicates('pubchem_cid')

In [185]:
column_to_move = df_comp.pop("pubchem_cid")
df_comp.insert(0, "pubchem_cid", column_to_move)
column_to_move = df_comp.pop("smiles")
df_comp.insert(1, "smiles", column_to_move)


In [186]:
df_sig_resp_ = df_sig_resp.merge(df_comp, on='pubchem_cid', how='left')

In [187]:
df_sig_resp_

,index,cl_sm_dose,cell_line,drug,dose,pubchem_cid,cell_line_drug,responses,smiles,fps_0,...,fps_2038,fps_2039,fps_2040,fps_2041,fps_2042,fps_2043,fps_2044,fps_2045,fps_2046,fps_2047
0,0,CVCL_0023_APTO-253_0.05,CVCL_0023,APTO-253,0.05,11960271,CVCL_0023_11960271,0.989992,CC1=C(C2=C(N1)C=CC(=C2)F)C3=NC4=C5C=CC=NC5=C6C...,0,...,0,0,0,0,0,0,0,0,0,0
1,1,CVCL_0023_AZD-7648_0.05,CVCL_0023,AZD-7648,0.05,135151360,CVCL_0023_135151360,0.991688,CC1=CC2=NC=NN2C=C1NC3=NC=C4C(=N3)N(C(=O)N4C)C5...,0,...,0,0,0,0,0,0,0,0,1,0
2,2,CVCL_0023_AZD-8055_0.05,CVCL_0023,AZD-8055,0.05,25262965,CVCL_0023_25262965,0.689011,CC1COCCN1C2=NC(=NC3=C2C=CC(=N3)C4=CC(=C(C=C4)O...,0,...,0,0,0,0,0,0,0,0,0,0
3,3,CVCL_0023_AZD1390_0.05,CVCL_0023,AZD1390,0.05,126689157,CVCL_0023_126689157,0.999031,CC(C)N1C2=C(C=NC3=CC(=C(C=C32)C4=CN=C(C=C4)OCC...,0,...,0,0,0,0,0,0,0,0,0,0
4,4,CVCL_0023_Afatinib_0.05,CVCL_0023,Afatinib,0.05,10184653,CVCL_0023_10184653,0.912780,CN(C)CC=CC(=O)NC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC(...,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234,7978,CVCL_1731_5-Fluorouracil_0.5,CVCL_1731,5-Fluorouracil,0.50,3385,CVCL_1731_3385,0.923447,C1=C(C(=O)NC(=O)N1)F,0,...,0,0,0,0,0,0,0,0,0,0
4235,7981,CVCL_1731_Belinostat_0.5,CVCL_1731,Belinostat,0.50,6918638,CVCL_1731_6918638,0.912687,C1=CC=C(C=C1)NS(=O)(=O)C2=CC=CC(=C2)C=CC(=O)NO,0,...,0,0,0,1,0,0,0,0,0,0
4236,7990,CVCL_1731_Fulvestrant_0.5,CVCL_1731,Fulvestrant,0.50,104741,CVCL_1731_104741,0.901148,CC12CCC3C(C1CCC2O)C(CC4=C3C=CC(=C4)O)CCCCCCCCC...,0,...,0,0,0,0,0,0,0,0,0,0
4237,7997,CVCL_1731_Panobinostat_0.5,CVCL_1731,Panobinostat,0.50,6918837,CVCL_1731_6918837,0.470417,CC1=C(C2=CC=CC=C2N1)CCNCC3=CC=C(C=C3)C=CC(=O)NO,0,...,0,0,0,0,0,0,0,0,0,0


In [188]:
df_comp

,pubchem_cid,smiles,fps_0,fps_1,fps_2,fps_3,fps_4,fps_5,fps_6,fps_7,...,fps_2038,fps_2039,fps_2040,fps_2041,fps_2042,fps_2043,fps_2044,fps_2045,fps_2046,fps_2047
0,387447,B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,25183872,B(C(CC(C)C)NC(=O)CNC(=O)C1=C(C=CC(=C1)Cl)Cl)(O)O,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,444972,C(=CC(=O)O)C(=O)O,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3657,C(=O)(N)NO,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11683005,C[As](C)SCC(C(=O)NCC(=O)O)NC(=O)CCC(C(=O)O)N,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,117947097,CS(=O)(=O)C1=C2C(C(C(C2=C(C=C1)OC3=CC(=CC(=C3)...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
155,24776445,CS(=O)(=O)C1=CC(=C(C=C1)C(=O)NC2=CC(=C(C=C2)Cl...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
156,2478,CS(=O)(=O)OCCCCOS(=O)(=O)C,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
158,5282451,C1C(C1)C2=NC3=CC=CC=C3C(=C2/C=C/[C@@H](O)C[C@...,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [191]:
df_sig_resp_final = df_sig_resp_.merge(df, on='cl_sm_dose', how='left')

,index,cl_sm_dose,cell_line,drug,dose,pubchem_cid,cell_line_drug,responses,smiles,fps_0,...,9985,9987,9988,9989,999,9990,9991,9992,9993,9997
0,0,CVCL_0023_APTO-253_0.05,CVCL_0023,APTO-253,0.05,11960271,CVCL_0023_11960271,0.989992,CC1=C(C2=C(N1)C=CC(=C2)F)C3=NC4=C5C=CC=NC5=C6C...,0,...,1.842929,0.366734,-0.335411,0.782379,-0.359100,-0.234669,2.092593,0.000000,-0.832242,1.058628
1,1,CVCL_0023_AZD-7648_0.05,CVCL_0023,AZD-7648,0.05,135151360,CVCL_0023_135151360,0.991688,CC1=CC2=NC=NN2C=C1NC3=NC=C4C(=N3)N(C(=O)N4C)C5...,0,...,2.075660,0.152147,0.160935,0.444318,-0.384877,-0.837646,2.344321,0.000000,-0.937542,0.649613
2,2,CVCL_0023_AZD-8055_0.05,CVCL_0023,AZD-8055,0.05,25262965,CVCL_0023_25262965,0.689011,CC1COCCN1C2=NC(=NC3=C2C=CC(=N3)C4=CC(=C(C=C4)O...,0,...,1.366750,0.400315,0.087598,0.663222,0.103934,-0.031691,2.111688,0.000000,-1.267154,0.513679
3,3,CVCL_0023_AZD1390_0.05,CVCL_0023,AZD1390,0.05,126689157,CVCL_0023_126689157,0.999031,CC(C)N1C2=C(C=NC3=CC(=C(C=C32)C4=CN=C(C=C4)OCC...,0,...,1.823357,-0.357533,-0.097746,0.487961,-1.207489,-0.640630,2.030661,0.000000,-1.128126,0.567049
4,4,CVCL_0023_Afatinib_0.05,CVCL_0023,Afatinib,0.05,10184653,CVCL_0023_10184653,0.912780,CN(C)CC=CC(=O)NC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC(...,0,...,1.965340,-0.081255,-0.490730,0.402371,-0.153901,-0.327424,2.319682,0.000000,-1.040558,0.416956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234,7978,CVCL_1731_5-Fluorouracil_0.5,CVCL_1731,5-Fluorouracil,0.50,3385,CVCL_1731_3385,0.923447,C1=C(C(=O)NC(=O)N1)F,0,...,1.684694,-0.625037,-1.846971,-0.312898,0.630495,-0.011337,0.936636,0.113086,-0.798684,0.454878
4235,7981,CVCL_1731_Belinostat_0.5,CVCL_1731,Belinostat,0.50,6918638,CVCL_1731_6918638,0.912687,C1=CC=C(C=C1)NS(=O)(=O)C2=CC=CC(=C2)C=CC(=O)NO,0,...,0.000000,-0.759764,-1.668616,-0.228991,-0.926570,-0.181311,1.156408,0.000000,-0.016793,0.744082
4236,7990,CVCL_1731_Fulvestrant_0.5,CVCL_1731,Fulvestrant,0.50,104741,CVCL_1731_104741,0.901148,CC12CCC3C(C1CCC2O)C(CC4=C3C=CC(=C4)O)CCCCCCCCC...,0,...,1.766649,-1.682095,-1.319940,-0.924046,-1.253834,-1.525262,1.179120,0.200925,0.015409,0.524940
4237,7997,CVCL_1731_Panobinostat_0.5,CVCL_1731,Panobinostat,0.50,6918837,CVCL_1731_6918837,0.470417,CC1=C(C2=CC=CC=C2N1)CCNCC3=CC=C(C=C3)C=CC(=O)NO,0,...,0.806367,-0.805154,-0.687915,-3.087473,-0.539735,1.091674,-1.113487,0.000000,0.600180,0.755800


In [200]:
pharm_comp = pd.read_parquet('../../NO_BACKUP/pharmacodb/compounds_pharmdb.parquet')

In [206]:
pharm_comp = pharm_comp[['inchikey', 'pubchem']].dropna()

In [209]:
df_lincs = df_lincs.merge(pharm_comp, left_on='inchi_key', right_on='inchikey')
#[['cell_drug']].drop_duplicates('cell_drug')

In [213]:
df_cell_drug = df_lincs['cellosaurus_id'] + '_' + df_lincs['pubchem']

In [224]:
lincs_cells = df_lincs['cellosaurus_id'].unique()

In [232]:
df_sig_resp_final[df_sig_resp_final['cell_line'].isin(lincs_cells)].to_parquet('../../NO_BACKUP/tahoe/final_tahoe_overlapping_lincs.parquet', index=False)

In [233]:
df_sig_resp_final[~df_sig_resp_final['cell_line'].isin(lincs_cells)].to_parquet('../../NO_BACKUP/tahoe/final_tahoe_no_overlappings.parquet', index=False)

In [4]:
df = pd.read_parquet('../../NO_BACKUP/tahoe/final_tahoe_no_overlappings.parquet')

In [12]:
df_lincs = pd.DataFrame(df_lincs).T.drop(columns=['sig_id', 'pert_id', 'cellosaurus_id','inchi_key','cell_drug','nearest_dose','responses','cmap_name', 'smiles'])

In [11]:
df = df.drop(columns=['index', 'cl_sm_dose', 'cell_line', 'drug', 'dose', 'pubchem_cid', 'cell_line_drug', 'responses','smiles'])

In [13]:
pd.concat([df, df_lincs])

,fps_0,fps_1,fps_2,fps_3,fps_4,fps_5,fps_6,fps_7,fps_8,fps_9,...,9985,9987,9988,9989,999,9990,9991,9992,9993,9997
0,0,0,0,0,0,0,0,0,0,0,...,1.861418,-1.113724,0.545364,-1.035998,-1.050404,-1.013388,-0.448658,0.0,0.854815,0.973242
1,0,0,0,0,0,0,0,0,0,0,...,1.701611,-1.2,0.757024,-1.07387,-2.592207,0.059973,-0.165119,0.0,1.028941,0.734825
2,0,0,0,0,0,0,0,0,0,0,...,0.605908,-0.97463,0.991741,-0.928049,-2.631717,-0.415128,-0.756783,0.0,1.175194,0.140158
3,0,1,1,0,0,0,0,0,0,0,...,1.202355,-1.610269,1.088643,-1.030269,-2.495177,-0.635733,-0.632332,0.0,1.110014,0.363301
4,0,0,0,0,0,0,0,0,0,0,...,0.379006,-1.431053,-0.713355,-1.472717,-0.487054,0.452793,-0.2501,0.0,0.337853,0.22394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3317,0,0,0,0,1,0,0,0,0,0,...,0.0,-0.759764,-1.668616,-0.228991,-0.92657,-0.181311,1.156408,0.0,-0.016793,0.744082
3318,0,0,0,0,0,0,0,0,0,0,...,1.766649,-1.682095,-1.31994,-0.924046,-1.253834,-1.525262,1.17912,0.200925,0.015409,0.52494
3319,0,0,0,0,0,0,0,0,0,0,...,0.806367,-0.805154,-0.687915,-3.087473,-0.539735,1.091674,-1.113487,0.0,0.60018,0.7558
3320,0,0,0,0,0,0,0,0,0,0,...,1.588909,-0.388464,-0.84587,0.372102,-0.5247,-0.14696,0.457382,0.0,-0.607064,0.457983
